In [1]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import csv
import os
import requests

In [ ]:
def getClassification():################获取全部图书分类信息
        resp=urlopen('http://category.dangdang.com/?ref=www-0-C')
        soup=BeautifulSoup(resp,'html.parser')
        ##############总目录
        General_classification =soup.find_all('div',class_='classify_kind_name')
        ##############分目录
        Subdivision_classification=soup.find_all('ul',class_='classify_kind_detail')
        sitename,siteurl=[],[]
        for i in range(31):
            sitename.append(site[i].a.string)
            siteurl.append(site[i].a.get('href'))
        sitenames,siteurls=[],[]
        for i in range(len(sitename)):  
                resp=urlopen('https://tianqi.moji.com'+siteurl[i])
                soup=BeautifulSoup(resp,'html.parser')
                shenghui=siteurl[i].split('/')
                shenghui=shenghui[3]
                site=soup.find_all(href=re.compile(shenghui+'/'))
                for _ in site:
                    sitenames.append(sitename[i]+_.string)
                    siteurls.append(_.get('href'))
    
        return sitenames,siteurls

In [18]:
resp=urlopen('http://category.dangdang.com/?ref=www-0-C')
soup=BeautifulSoup(resp,'html.parser')
##############总目录
General_classification =soup.find_all('div',class_='classify_kind_name')
##############分目录

In [14]:
a.encode('iso-8859-1').decode('gb2312')

'青春文学'

In [42]:
for i in General_classification:
    try:
        print(i.a.string.encode('iso-8859-1').decode('gbk')
    except:
        print('港台图书')
    

SyntaxError: invalid syntax (<ipython-input-42-fcae228053b5>, line 4)